In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib

import matplotlib.pyplot


# Correlations

#### Observe corellations for all makes

import seaborn as sn
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (19.20,10.80))

corrMatrix = df.corr()
sn.heatmap(corrMatrix, annot=True)
plt.savefig('visualizations/sedan_total_corr.png')

When observed as a whole, the corellations between certain features are below significancee

when each make is split into a group, the correlations are more easily observed

make_list = df['make'].unique()

for i, make in enumerate(make_list):
    df_new = df.loc[df['make']==f'{make}']
    df_new = df_new.drop(columns=['makeID'])
    fig = plt.figure(figsize = (19.20,10.80))

    corrMatrix = df_new.corr()
    sn.heatmap(corrMatrix, annot=True)
    plt.title(f'{make}')
    plt.savefig(f'visualizations/Sedan_Correlations/sedan_{make}_corr.png')
    plt.close()

with the overall data giving un-usable correlations we may need to have a regression model for each make/body class?

Test the difference between a prediction using the total data and a single make, data.

---------------------

# Random Forest Regrssion

-----------

In [34]:
df = pd.read_csv('csv/SEDAN_DATA_CLEAN.csv')
df

,brandName,modelName,vf_ModelID,vf_ModelYear,mileage,askPrice,msrp,color,vf_EngineCylinders,vf_FuelTypePrimary,vin
0,FORD,Taurus,1782.0,2015.0,67588,16462,16462,Ingot Silver Metallic,6.0,Gasoline,85384fd9108f6e9c75d7a538e4ce8a892170f7dbf42264...
1,CHRYSLER,300,1878.0,2017.0,9231,23962,23962,Billet Silver Metallic Clearcoat,6.0,Gasoline,5c45a1254ea832cffc329eebf700acf918682c1b40f9a7...
2,LINCOLN,MKZ,1790.0,2016.0,19710,22962,22962,White Platinum Metallic Tri-Coat,6.0,Gasoline,95592374eecca29c4614b5987b760f9b528e6d3e33c2db...
3,FORD,Fiesta,3267.0,2011.0,92053,5962,5962,Blue,4.0,Gasoline,5558ab8c6833c13904fa4a1fbd45c8d00d02f6db9e53fe...
4,INFINITI,G37,2337.0,2013.0,74917,13962,13962,Vibrant Red,6.0,Gasoline,10e990a77a9615aca1f0e2210513a289fe54a9175a522d...
...,...,...,...,...,...,...,...,...,...,...,...
138313,MERCEDES-BENZ,C-Class,2085.0,2016.0,18322,23999,23999,Polar White,4.0,Gasoline,a5edaa50f3a74293f38ba7611f5be5db8bede213acdf96...
138314,LINCOLN,Town Car,1791.0,2004.0,135839,5900,5900,Medium Steel Blue Clearcoat Metallic,8.0,Gasoline,04a3a8af9f12eaa2b8466dd3588994cfbf333d30bf9eee...
138315,CHEVROLET,Cruze,1832.0,2011.0,98007,5950,5950,Imperial Blue Metallic,4.0,Gasoline,b31d181e2ba2f8a7b56a353a81a55951ad833ab2ea5180...
138316,HONDA,Accord,1861.0,2015.0,44717,18500,18500,Crystal Black Pearl,4.0,Gasoline,6afcb285f492dcb542c720b50f8fa32e57d8fc43ea95b2...


df = df.rename(columns={
    'brandName': 'brandName',
    'modelName': 'modelName',
    'vf_ModelID': 'modelID',
    'vf_ModelYear': 'modelYear',
    'mileage': 'mileage',
    'askPrice': 'askPrice',
    'msrp': 'msrp',
    'color': 'color',
    'vf_EngineCylinders': 'engineCylinders',
    'vf_FuelTypePrimary': 'fuelTypePrimary',
    'vin': 'vin'
})

In [47]:
def RF_regr_alldata(depth):
    model_results = {}
    train_score_list = []
    test_score_list = []
    df_test = df.drop(columns=['msrp','vin'])
    #names for each DF/ Model to be trained


    # divide the data
    X = df_test.drop(columns=['askPrice'])
    y = df_test['askPrice']

    # set dummies
    X_dummies = pd.get_dummies(X,drop_first=True)

    # split for test and train
    X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, random_state=42)

    # model parameters
    model = RandomForestRegressor(max_depth=depth,n_estimators=100 random_state=12)

    # fit the model to the data
    model.fit(X_train, y_train)

    # evaluate scores and predictions
    train_predicted = model.predict(X_train)
    test_predicted = model.predict(X_test)

    train_score = model.score(X_train,y_train,sample_weight=None)
    test_score = model.score(X_test,y_test,sample_weight=None)

    # record results

    model_results['model'] = model

    model_results['train_y_pred'] = train_predicted
    model_results['test_y_pred'] = test_predicted

    model_results['train_score']  = train_score
    model_results['test_score']  = test_score

    train_score_list.append(train_score)
    test_score_list.append(test_score)
    return model_results, train_score_list, test_score_list

In [48]:
model_results, train_score_list, test_score_list = RF_regr_alldata(7)

In [49]:
model_results

{'model': RandomForestRegressor(max_depth=4, n_estimators=300, random_state=12),
 'train_y_pred': array([16830.73651763, 16830.73651763, 24626.31127783, ...,
        16830.73651763, 16830.73651763, 12589.714009  ]),
 'test_y_pred': array([19216.94316023,  9935.82836627,  8941.53607819, ...,
        12586.76204345, 16830.73651763, 16830.73651763]),
 'train_score': 0.6452143568065256,
 'test_score': 0.6518323138989384}

In [45]:
model = model_results['model']
joblib.dump(model,'models/sedan_depth_7_81.joblib')

['models/sedan_depth_7_81.joblib']

In [37]:
model = joblib.load('models/sedan_depth_5_72.joblib')

In [39]:
df_test = df.drop(columns=['msrp','vin'])


X = df_test.drop(columns=['askPrice'])
y = df_test['askPrice']

# set dummies
X_dummies = pd.get_dummies(X,drop_first=True)

# split for test and train
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, random_state=42)


In [40]:
model.predict(X_test)

array([17993.29477626,  8984.14488702,  9615.43675097, ...,
       12685.99833404, 16205.13847434, 16205.13847434])

In [42]:
model_results['test_y_pred']

array([17993.29477626,  8984.14488702,  9615.43675097, ...,
       12685.99833404, 16205.13847434, 16205.13847434])